<a href="https://colab.research.google.com/github/probabll/ntmi-tutorials/blob/main/T3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guide

* Check the entire notebook before you get started, this gives you an idea of what lies ahead.
* Note that, as always, the notebook recaps theory, and contains solved exercises. While you should probably make use of this theory recap, be careful not to spend disproportionately more time on this than you should. The theory here is more condensed, and should be easier to understand after week 3's reading and after the highlights discussed in class (HC3a).
* We recommend you read the theory part before the LC3 session.

## ILOs

After completing this lab you should be able to 

* develop generalised linear models of text classification and text regression in Python using sklearn and jax
* estimate parameters via gradient-based optimisation

**General notes**

* In this notebook you are expected to use $\LaTeX$. 
* Use python3


## Table of Contents

* Setting up
* Theory
    * Feature functions
    * Generalised linear models
        * Bernoulli GLM
        * Categorical GLM
        * Poisson GLM
    * General principles for prescribing GLMs
    * Parameter estimation
        * Stochastic gradient-based optimisation
        * Regularisation
* Binary classifier
    * Naive Bayes
    * Binary GLMs on Jax
* Python/Jax GLM class
    * Binary classifier experiment
    * Poisson regressor experiment

## Table of graded exercises

Exercises have equal weight.

* [Subjectivity classifier](#ex-bernoulli)
* [Number of Tokens Data and Baseline](#ex-numtokens)
* [Poisson regression](#ex-poisson)

# Setting up

Make sure you have colab configure to use 4 spaces for TAB (Tools/Settings/Editor/Indentation). If you change your Runtime to GPU (Runtime/Change runtime type) model training will be faster, but for this tutorial a GPU is not strictly necessary.

In [ ]:
!pip install numpy
!pip install jax
!pip install sklearn
!pip install pandas
!pip install seaborn
!pip install tabulate

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import jax
from jax import device_put

import json
import gzip
import urllib.request

from tabulate import tabulate

import sklearn
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

import nltk

nltk.download('punkt')
nltk.download('toktok')
nltk.download('sentence_polarity')
nltk.download('subjectivity')
nltk.download('brown')

# Theory

We often need to analyse documents in terms of diverse properties. Where these properties are discrete and finite (e.g., negative/neutral/positive, or spam/no-spam, or politics/sport/science) we talk about text analysis as **text classification** (or text categorisation), where these properties are numerical we talk about text analysis as **text regression**. 

For the rest of this notebook, we use $X$ to denote a random document in the space $\mathcal X$ of all possible documents. A document for us is a sequence $x=\langle w_1, \ldots, w_l\rangle$ of $l = |x|$ words, where each word belongs to a vocabulary $\mathcal W$ of $V = |\mathcal W|$ words. We use $y \in \mathcal Y$ to denote a target (or response) variable, which will be from a countably finite set $\{1, \ldots, K\}$ in text classification, or from a countably infinite set (e.g., $\mathbb N_0$ or $\mathbb N_1$) or an uncountable set (e.g., $\mathbb R$, $\Delta_{3-1}$) in text regression.

## Feature Functions

Before we can talk about our key modelling idea, we need to talk about feature functions. 

A **vector-valued feature function** 
\begin{equation}
  \mathbf h: \mathcal X \to \mathbb R^D
\end{equation}
maps a document $x \in \mathcal X$ to a $D$-dimensional *feature vector*. The feature vector is something that "describes" a document along $D$ numerical dimensions, each of which quantifies an aspect of the problem that's believed to play some significant role in the kind of analysis we are making. 

In this notebook we will use feature functions based on count vectors. The simplest such feature function has a dimension for each known word (i.e., $D=V$), the $\mathbf(x)$ is such that $h_d(x) = \sum_{i=1}^l [x_i = d]$ is the number of times the word corresponding to dimension $d$ occurs in $x$. We may also normalise this by sequence length, in that case, $h_d(x) = \frac{1}{l}\sum_{i=1}^l [x_i = d]$ is one definitoin of *term-frequency* (or tf). In some applications it also makes sense to discount the importance of words based on whether they are themselves too frequent to be informative, this can be achieved by the inverse-document-frequency (or idf). For count-based, tf-based, and tf-idf-based features we will use robust implementations from [`scikitlearn`](https://scikit-learn.org/stable/modules/feature_extraction.html). 

## Generalised Linear Models

Our new general tool for text analysis (both classification and regression) is a *generalised linear model* (GLM). 

A GLM is a conditional model of $Y|X=x$ where we compute the *parameter* of our statistical model with the help of a parametric linear transformation of $\mathbf h(x)$. The linear transformation has its own parameters (the coefficients we multiply and the biases we add), which we will denote generically by $\boldsymbol\theta$. 

A GLM is a conditional model of $Y|X=x$, as always, the conditional is governed by a known parametric family that we get to choose (e.g., Bernoulli, Categorical, Binomial, Geometric, Poisson, Zeta, etc.). To fully specify a model, we need to predict from $x$ the statistical parameter of the distribution, let's denote that parameter by $g(x; \boldsymbol\theta)$, where $\boldsymbol\theta$ is the collection of trainable parameters of the model.

In a GLM, $g(x; \boldsymbol\theta)$ is a (possibly nonlinear) transformation $a(\cdot)$ of a linear function of $\boldsymbol\theta$ and $\mathbf h(x)$. That is:  
\begin{equation}
g(x; \boldsymbol\theta) = a(\underbrace{\mathbf w^\top \mathbf h(x) + b)}_{\text{1 linear predictor}}
\end{equation}
with $\mathbf w \in \mathbb R^D$ and $b \in \mathbb R$, for a single-parameter distribution; or
\begin{equation}
\mathbf g(x; \boldsymbol\theta) = \mathbf a(\underbrace{\mathbf W^\top \mathbf h(x) + \mathbf b}_{K\text{ linear predictors}})
\end{equation}
with $\mathbf W \in \mathbb R^{D \times K}$ and $\mathbf b \in \mathbb R^K$, for a $K$-parameters distribution.

The function $a(\cdot)$ is used to constrain the quantity  $\mathbf w^\top \mathbf h(x) + b$, also called **linear predictor**, to the valid range of parameters for the statistical distribution we chose. This function is called *inverse link function* in statistics, and it is called **activation function** in deep learning.

In the multiparameter case, the activation function $\mathbf a(\cdot)$ is vector-valued. This tutorial will mostly concentrate on the single parameter case, which is sufficient for most applications of binary classification and ordinal regression.

### Bernoulli GLM

For example, in spam classification we want to map a document $x \in \mathcal X$ to the probability of it being spam or not, let's denote this probability by $g(x; \boldsymbol\theta)$ and recall that $0 \le g(x; \theta) \le 1$. 

The statistical model of interest is
\begin{align}
Y|X=x \sim \mathrm{Bernoulli}(g(x; \boldsymbol\theta))
\end{align}
where $g(x; \boldsymbol\theta)$ is the *probability* of labelling a document as spam. 

Let's define this function $g$. As documents are not objects in a numerical space, our GLMs will all start by mapping $x$ to a feature vector via a feature function $\mathbf h(x)$. One possible GLM for this binary classification problem is the following:
\begin{align}
  g(x; \boldsymbol\theta) &= \mathrm{sigmoid}(\mathbf w^\top \mathbf h(x) + b)
\end{align}
where $\mathbf w \in \mathbb R^{D}$ and $b \in \mathbb R$ are the parameters of the model $\boldsymbol\theta = \{\mathbf w, b\}$.
Step by step:
1. we map $x \in \mathcal X$ to a feature vector $\mathbf h(x) \in \mathbb R^D$; this is a deterministic step and it does not require interacting with the parameters $\boldsymbol\theta$ of the GLM;
2. we take the dot-product between the feature vector and the parameter $\mathbf w$, which gives us a scalar (a real value);
3. we add a bias value to that scalar, obtaining a scalar (a real value);
4. finally, the [sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function) maps this real value to a valid probability value.





**Exercise with solution** Plot the sigmoid function for the following range of values:

```python
s = np.linspace(-10, 10, 1000)
```

<details>
    <summary><b>Solution</b></summary>

```python
# **SOLUTION**
_ = plt.plot(np.linspace(-10, 10, 1000), 1/(1 + np.exp(-np.linspace(-10, 10, 1000))))
_ = plt.xlabel('s')
_ = plt.ylabel('1/(1+exp(-s))')
```

---    
</details>


### Categorical GLM

This idea is so powerful that we can build pretty much every text classifier and every text regressor in the book. 

For example, for a $K$-way classifier 
\begin{align}
  Y|X=x \sim \mathrm{Categorical}(\mathbf g(x; \boldsymbol\theta))
\end{align}
here $\mathbf g(x; \boldsymbol\theta) \in \Delta_{K-1}$ is a $K$-dimensional vector of positive values that sum to 1. 
This is a valid GLM for this model:
\begin{align}
\mathbf s &= \mathbf W^\top\mathbf h(x) + \mathbf b \\
\mathbf g(x; \boldsymbol\theta) &= \mathrm{softmax}(\mathbf s)
\end{align}

where $\boldsymbol = \{\mathbf W, \mathbf b\}$ are the parameters of the model, $\mathbf W \in \mathbb R^{D \times K}$ is a matrix that maps $D$ inputs (the feature values) to $K$ outputs (real values), $\mathbf b \in \mathbb R^K$ is a vector of biases (one real value bias per class), and [softmax](https://en.wikipedia.org/wiki/Softmax_function) turns the $K$-dimensional linear predictor $\mathbf s$ into $K$ strictly positive numbers that sum to 1 (i.e., a probability vector in the simplex $\Delta_{K-1}$).

In deep learning literature the vector $\mathbf s$ that we use as input to the softmax function is often called the vector of *scores* or *logits*, each one of its coordinates quantifies the importance of the corresponding class in a logarithmic scale.

This model is also known as a *log-linear* model, that is because the logarithm of the softmax is a linear function of $\mathbf h(x)$ and $\boldsymbol\theta$.


Here we demonstrate that the softmax function indeed maps 3-dimensional real vectors to 3-dimensional probability vectors.

In [ ]:
def np_softmax(x, axis=-1):
    """Compute softmax values for each sets of scores in x."""
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

# let's get 1000 values for s0
s0 = np.linspace(-3.0, 3.0, 1000)
# for s1 and s2 we will have 1s and 0s
s = np.vstack([s0, np.ones_like(s0), np.zeros_like(s0)]).T

_ = plt.plot(s0, np_softmax(s)[:, 0], linewidth=2, label='output 0')
_ = plt.plot(s0, np_softmax(s)[:, 1], linewidth=2, label='output 1')
_ = plt.plot(s0, np_softmax(s)[:, 2], linewidth=2, label='output 2')
_ = plt.plot(s0, np_softmax(s).sum(-1), ':', c='black', linewidth=2, label='elementwise sum')
_ = plt.xlabel(r'$s_0$')
_ = plt.ylabel('softmax output')
_ = plt.legend()

### Poisson GLM

Let's attempt to parameterise a conditional distribution over the number of likes $Y$ that a tweet $x$ will receive, based on its content as captured by a feature function $\mathbf h(x) \in \mathbb R^D$.

Suppose that for this statistical model, we decide to use a Poisson distribution, then
\begin{align}
  Y|X=x &\sim \mathrm{Poisson}(g(x; \boldsymbol\theta))
\end{align}
because this is a Poisson, we now that $g(x; \boldsymbol\theta)$ must be strictly positive. So, here's a valid GLM:
\begin{align}
  g(x; \boldsymbol\theta) &= \exp(\mathbf w^\top \mathbf h(x) + b)
\end{align}
with model parameters are $\boldsymbol\theta = \{\mathbf w, b\}$ with $\mathbf w \in \mathbb R^D$ and $b \in \mathbb R$.

Another GLM for the Poisson is 
\begin{align}
  g(x; \boldsymbol\theta) &= \mathrm{softplus}(\mathbf w^\top \mathbf h(x) + b)
\end{align}
which uses the softplus function $\log(1 + e^s)$. The softplus function, like the exp, turns any real number into a strictly positive number. Unlike the exp, the softplus is approximately linear as $s$ increases.


**Exercise with solution** Plot the both the exponential and the softplus function over the following range of values:

```python
s = np.linspace(-10, 10, 1000)
```

Use subplots so you can visualise hwo they differ for larger positive $s$.

<details>
    <summary><b>Solution</b></summary>

```python
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
_ = axs[0].plot(np.linspace(-10, 10, 1000), np.exp(np.linspace(-10, 10, 1000)))
_ = axs[0].set_xlabel('s')
_ = axs[0].set_ylabel('exp(s)')
_ = axs[1].plot(np.linspace(-10, 10, 1000), np.log(1 + np.exp(np.linspace(-10, 10, 1000))))
_ = axs[1].set_xlabel('s')
_ = axs[1].set_ylabel('softplus(s)')
fig.tight_layout(h_pad=1, w_pad=1)
```

---    
</details>


## General principles for prescribing GLMs

1. Start with choosing  the family you will model with. This depends on the type of data you have (e.g., binary means Bernoulli, $K$-way classification usually means Categorical, ordinal regression usually means Poisson, but it could be some other distribution over natural numbers or a subset thereof, like the Binomial, continuous regression might require a Normal distribution, regressing to vectors of proportions might require a Dirichlet distribution, etc.). You don't need to know all these distributions by heart, when needed, we will give you information about them that will help you judge their relevance in context.

2. The input is text, but GLMs operate with inputs in the real coordinate space, so you need a vector-valued feature function $\mathbf h(x)$.

3. In a GLM, the input $\mathbf h(x)$ and the parameters $\boldsymbol\theta$ interact linearly. This constrains us to either operations like dot product, matrix multiplication and scalar or vector addition. Whether we have "dot product plus scalar" or "matrix multiplication plus vector" depends exclusively on the dimensionality we need for the linear predictor. If we need a single scalar, we wil use the former. If we need a vector, we will use the latter.

4. Example 1: the Poisson parameter is a single scalar, thus we know that we need to map from $\mathbf h(x)$ to a single scalar, as we achieve with "dot product plus scalar". Example 2: the Categorical parameter is a vector, thus we know that we need to map from $\mathbf h(x)$ to a $K$-dimensional vector, as we achieve with "matrix multiplication plus vector".

5. Finally, the statistical parameter is generally constrained to a subset of the real numbers, so we need an activation function that constrains the linear predictor accordingly. Example 1: the Poisson parameter is *strictly positive* by definition, so we need to wrap the linear function around something whose output is *never negative* and *never 0*, no matter which real-valued input we give it. The exponential function does that for us. There are other activation functions that achieve the same result, but the exponential is convenient for certain reasons (e.g., it's logarithm is linear). In our implementation below we will see other options. Example 2: the Categorical parameter must be a probability vector, the softmax function can realise that constraint for us.



## Parameter estimation

Given a training set $\mathcal D = \{(x^{(n)}, y^{(n)})\}_{n=1}^N$ of $N$ observed input-target pairs, we would ideally choose the parameter vector $\boldsymbol\theta$ that maximises the log-likelihood of the model:

\begin{align}
\mathcal L_{\mathcal D}(\boldsymbol\theta) &= \sum_{n=1}^N \log P(Y=y^{(n)}|X=x^{(n)}; \boldsymbol \theta)  \\
\boldsymbol\theta^\star &= \arg\max_{\boldsymbol\theta}~\mathcal L_{\mathcal D}(\boldsymbol\theta) 
\end{align}

Unlike tabular CPDs, there is no simple expression for the MLE of a GLM. But, we can employ a gradient-based search. This search uses the gradient $\nabla_{\boldsymbol \theta}\mathcal L_{\mathcal D}(\boldsymbol\theta)$ to iteratively update an existing $\boldsymbol \theta$, starting from an initial guess $\boldsymbol \theta^{(0)}$, which is typically a random initialisation of the parameters.

At iteration $t$, the update rule is
\begin{equation}
\boldsymbol \theta^{(t+1)} = \boldsymbol \theta^{(t)}  + \gamma_t \nabla_{\boldsymbol \theta^{(t)}}\mathcal L_{\mathcal D}(\boldsymbol\theta^{(t)})
\end{equation}
where the log-likelihood is assessed using the current parameters, we obtain the gradient for it, and combine it with the current parameters to get the next iterate. The quantity $\gamma_t > 0$ is called a *learning rate*.

<details>
    <summary><b>Maximisation vs minimmisation</b></summary>

If you have seen this formula before with a *minus* rather than a *plus* for the gradient, don't worry, it is the same notion. You *sum* the gradient if you are maximising the log-likelihood, and you *subtract* the gradient if you are minimising the negative log-likelihood. The two procedures yield the exact same optimum.

---    
</details>


### Stochastic gradient-based optimisation

Assessing each one of the terms of the log-likelihood function 

Assessing the log-likelihood of a certain value of the parameter vector $\boldsymbol\theta$ requires assessing the probability mass (or density, for continuous variables) of each one of our observations under the current value of the parameter. Each one such assessment on its own is not at all challenging, but assessing all the $N$ terms can be challenging for large $N$.

Fortunately, we can use a stochastic gradient procedure, which still has the same guarantees as the deterministic procedure.

At each iteration $t$, we compute an approximation to $\nabla_{\boldsymbol \theta^{(t)}}\mathcal L_{\mathcal D}(\boldsymbol\theta^{(t)})$. This approximation is a Monte Carlo (MC) estimate obtained using $S < N$ data points uniformly sampled from $\mathcal D$:

\begin{align}
\nabla_{\boldsymbol \theta^{(t)}}\mathcal L_{\mathcal D}(\boldsymbol\theta^{(t)}) &\overset{\text{MC}}{\approx} \frac{1}{S} \sum_{n=1}^S \nabla_{\boldsymbol \theta^{(t)}} \log P(Y=y|X=x; \boldsymbol \theta^{(t)})
\end{align}

We can obtain this gradient by essentially pretending, at each iteration $t$, that the log-likelihood function depends only on a small *batch* of $S$ observations drawn from the training set:

\begin{equation}
\mathcal L_{\mathcal B}(\boldsymbol\theta^{(t)}) = \frac{1}{S}\sum_{s=1}^S \log P(Y=y^{(s)}|X=x^{(s)}; \boldsymbol \theta^{(t)})
\end{equation}


### Regularisation

Oftentimes, we have **many** features, and thus **many parameters**. This gives models the capacity to discover correlations that have no real predictive power (e.g., that [capivaras](https://en.wikipedia.org/wiki/Capybara) implies a *negative* sentiment, simply because the 1 time that word was seen in the data was in a document labelled with the negative class, for example the document might have been "I loved the capivaras but overall the horrible weather ruined the trip"). 

These are called **spurious correlations** and we would rather not be mislead by them. 

In an attempt to get rid of them, we employ a so called **regulariser**. This is a penalty on the objective based on the norm of the parameter vector, and usually employ the L2 norm.

The L2 norm of a vector is:
\begin{equation}
\mathcal R(\mathbf v) = \sqrt{\sum_{d=1}^D v_d^2}
\end{equation}
For a collection of parameter vectors we sum the L2 norms of each vector. 

Thus, the final objective for *regularised maximum likelihood estimation* is 

\begin{equation}
\boldsymbol\theta^\star = \arg\max_{\boldsymbol\theta}~\mathcal L_{\mathcal D}(\boldsymbol\theta) - \lambda \mathcal R(\boldsymbol\theta)
\end{equation}
where $\lambda \ge 0$ is a hyperparameter used to control the importance of the regulariser. There's no way to choose the value of $\lambda$ directly from the training data, we have to try various values and test the model's performance on heldout data.

# Binary classifier

Here we use the `subjectivity` corpus from NLTK to train binary classifiers. We will train a Naive Bayes classifier using scikitlearn and a Bernoulli GLM which we will develop in JAX.

In [ ]:
from nltk.corpus import subjectivity  # binary classification

In [ ]:
labels = subjectivity.categories()
C = len(labels)
print("{}-way classification:\n{}".format(C, '\n'.join(labels)))

As usual, we will split our observations in three disjoint sets, 80% for training, 10% for whatever development purposes we have, and 10% for testing the generalisation of our classifier at the end. 

In [ ]:
import numpy as np


def prepare_nltk_corpus(nltk_corpus, categories, seed=23, BOS='<s>', EOS='</s>'):
    """
    Prepare an nltk text categorization corpus in a sklearn friendly format.
    
    This function is very similar to what you saw in T2, but here we add BOS tokens in addition to EOS tokens 
    (while the BOS token has no effect in NBC with unigram conditionals, 
    it can be useful for some of the feature-richer classifiers we will develop here).
    
    :param nltk_corpus: something like sentence_polarity
    :param categories: a list of categories (each a string), 
        sklearn will treat categories as 0-based integers, thus we will map the ith element in this list to y=i
    :param seed: for reproducibility
    :param BOS: if not None, start every sentence with a single BOS token
    :param EOS: if not None, end every sentence with a single EOS token
    :return: training, dev, test
        each an np.array such that 
        * array[:, 0] are the inputs (documents, each a string)
        * array[:, 1] are the outputs (labels)
    """
    pairs = []    
    prefix = [BOS] if BOS else []
    suffix = [EOS] if EOS else []
    for label in categories:  # here we pair doc (as a single string) and label (string)
        # this time we will concatenate the EOS symbol to the string
        pairs.extend((' '.join(prefix + s + suffix), label) for s in nltk_corpus.sents(categories=[label]))
    # we turn the pairs into a numpy array
    # np arrays are very convenient for the indexing tools np provides, as we will see
    pairs = np.array(pairs)
    # it's good to shuffle the pairs
    rng = np.random.RandomState(seed)    
    rng.shuffle(pairs)
    # let's split the np array into training (80%), dev (10%), and test (10%)
    num_pairs = pairs.shape[0]
    # we can use slices to select the first 80% of the rows
    training = pairs[0:int(num_pairs * 0.8),:]
    # and similarly for the next 10%
    dev = pairs[int(num_pairs * 0.8):int(num_pairs * 0.9),:]
    # and for the last 10%
    test = pairs[int(num_pairs * 0.9):,:] 
    return training, dev, test

Separate our corpus into training/dev/test sets:

In [ ]:
so_training, so_dev, so_test = prepare_nltk_corpus(subjectivity, labels)

In [ ]:
so_training.shape, so_dev.shape, so_test.shape

**Exercise with solution** As always, begin by inspecting your data.

<details>
    <summary><b>Solution</b></summary>

```python

# **SOLUTION**

_ = plt.hist([len(x.split()) for x, y in so_training], bins='auto')
_ = plt.xlabel("Length in tokens")
_ = plt.show()

_ = plt.hist([y for x, y in so_training], bins='auto')
_ = plt.ylabel("Frequency")
_ = plt.xlabel("Class")
_ = plt.show()

print(tabulate(so_training[4:6], headers=['doc', 'label']))
```

---    
</details>


Helper code to map named labels to 0-based integers and back:

In [ ]:
def label_as_string(y, vocab={True: 'subj', False: 'obj', 1: 'subj', 0: 'obj'}):
    """Map from boolean or integer to a string label"""
    return [vocab[b] for b in y]

def label_as_int(y, vocab={'subj': 1, 'obj': 0}):
    """Map from string to integer"""
    return np.array([vocab[b] for b in y], dtype=int)  

In [ ]:
label_as_string([True, False]), label_as_int(['subj', 'obj'])

## Naive Bayes 

Let's start with a classifier we already know, this will also help us get comfortable with feature functions. 

The NBC can be represented in terms of feature functions. Suppose a feature function $\mathbf h(x)$ maps $x$ to a space where each coordinate $d$ corresponds to a word in the vocabulary, and $h_d(x)$ is the number of times that word occurs in $x$. This would make $\mathbf h(x) \in \mathbb R^V$ a $V$-dimensional vector, but most of its coordinates would be in fact 0, since only up to $l = |x|$ words occur in $x$. If we use the notation $(f, n) \in \mathbf h(x)$ to denote all feature-count pairs for which the feature is a word and the count is not zero, we can rewrite the joint probability of the NB classifier as follows:

\begin{equation}
    P_{YX}(y, x) = P_Y(y)\prod_{(f, n) \in \mathbf h(x)} P_{F|Y}(f|y)^n
\end{equation}

In class we discussed how this view of NBC can be used to generalise it to feature types are not just words, but in this tutorial we will continue working with word counts.

To represent feature functions efficiently sklearn uses *vectorizers*. These classes turn a string into a sparse vector of coded features. Internally this builds a vocabulary of features and a data structure that is sparse like nested dicts, but much more efficient.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Take a moment to read the documentation of `CountVectorizer?` and play a bit with some examples.

The count vectorizer can be used to implement NBC. We simply need to count the unigrams, which in sklearn style is done as follows:

In [ ]:
toy_vectorizer = CountVectorizer(ngram_range=(1,1), min_df=2)
# let's start with the first 5 sentences in the training set, just to see what this does
toy_vectorizer.fit(so_training[:5, 0])  
print(tabulate(so_training[:5], headers=['doc', 'label']))
print(f"The feature space contains {len(toy_vectorizer.get_feature_names_out())} features.\nThey are:")
for fname in toy_vectorizer.get_feature_names_out():
    print(f" F={fname}")
print("Note that skleanr is pre-processing the text for us, getting rid of some punctuation marks and English stopwords. We also chose to keep only words that occurred at least twice (with min_df=2).")
print("To apply the feature function to text we use the method *transform*:")
h_x05 = toy_vectorizer.transform(so_training[:5, 0])
print(f" which gives us an object of shape {h_x05.shape}, as expected.")
print(h_x05.shape)
print("But that object is a sparse array (the zeros are not stored in it):")
print(h_x05)
print("In some occasions we may need a dense numpy array, when that's the case we can use *toarray*:")
print(h_x05.toarray())

With count features we can implement the NB classifier. In sklearn it is called `MultinomialNB`. 

**Exercise with solution** Check its documentation. Train a CountVectorizer with `min_df=1` using all of the training data. Then use it to train a MultinomialNB with alpha (the Laplace smoothing coefficient) set to 0.7. Evaluate its performance on the *test set*, report the results via sklearn's `classification_report` as well as `ConfusionMatrixDisplay.from_predictions`.

You should obtain about 92% macro F1.

This tutorial is *not* about NBC, we train it just for comparison and to teach you a nice sklearn trick. Don't let this exercise take too much of your time.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

<details>
    <summary><b>Solution</b></summary>

```python
# **SOLUTION**
nb_vectorizer = CountVectorizer(ngram_range=(1,1))
nb_vectorizer.fit(so_training[:, 0])
nb_cls = MultinomialNB(alpha=0.7)
nb_cls.fit(nb_vectorizer.transform(so_training[:,0]), so_training[:, 1])

print("Classification report")
y_pred = nb_cls.predict(nb_vectorizer.transform(so_test[:, 0]))
print(classification_report(so_test[:, 1], y_pred))
print("Confusion matrix")
_ = ConfusionMatrixDisplay.from_predictions(so_test[:,1], y_pred)
```

---    
</details>


<details>
    <summary><b>Curious how we got to alpha=0.7?</b></summary>

We got sklearn to perform a grid search using cross validation on the training set for us:

```python
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

cls_nb = Pipeline(
    [
        ('vect', CountVectorizer(ngram_range=(1,1))),
        ('clf', MultinomialNB(alpha=0.7)),                 
    ]
)
# See the hyperparameters of the model (stuff like ngram_range and Laplace coefficient)
print(cls_nb.get_params())
# We can search for values of the hyperparameters of interest using cross validation
nb_grid = GridSearchCV(cls_nb, param_grid={'clf__alpha': np.linspace(0.1, 1., 10)}, cv=3)
nb_grid.fit(so_training[:, 0], so_training[:, 1])
print(nb_grid.best_params_)
print(classification_report(so_dev[:,1], nb_grid.predict(so_dev[:, 0])))
```

---    
</details>


## Binary GLMs on Jax

To train GLMs we need gradient-based optimisation. Nowadays we do not compute derivatives by hand, nor implement automatic differentiation, we use software that does that for us, efficiently and realiably. 

In this tutorial you will use JAX, for you've already seen it in ML.

Because we need JAX's automatic differentiation, we will need to express our model's computations using building blocks available in JAX. Luckily for us, JAX implements something very much like numpy and something very much like scipy, so all we've been using so far in the course is likely already available in JAX.

In [ ]:
import jax.numpy as jnp
import jax.scipy as jsp
from jax import grad, value_and_grad, random
from jax.nn import softplus, softmax, sigmoid

In [ ]:
# we need a random generator in order to create arrays in JAX
key = random.PRNGKey(0)

Here we will 

1. describe the necessary functions to parameterise a Bernoulli GLM
2. implement a stochastic gradient-based procedure for parameter estimation.

Roughly this is what we need to do:

1. obtain a feature function $\mathbf h$ and compute it for all our data points
2. obtain some initial parameters for the linear function
3. implement the linear function that outputs the linear predictor
4. constrain the linear predictor to being a probability value
5. write a training loop, where we use some subset of the data to estimate the value of the log-likelihood function, use the gradient of that value with respect to model parameters to improve our parameter estimates

So, let's get started. 

### Feature function

For feature function we are going to start with word counts (ie, `CountVectorizer`) and then we will normalise those into `tf-idf` features (see `TfidfTransformer`), which will help us take frequency information into account.

In [ ]:
so_ff = Pipeline(
    [
        ('vect', CountVectorizer(ngram_range=(1,1), min_df=5)),
        ('tfidf', TfidfTransformer()),
    ]
)
so_ff.fit(so_training[:, 0], so_training[:, 1])

In [ ]:
so_training_h_sparse = so_ff.transform(so_training[:, 0])
so_dev_h_sparse = so_ff.transform(so_dev[:, 0])
so_test_h_sparse = so_ff.transform(so_test[:, 0])

In [ ]:
# The number of input features is the $D$ in the formulas above.
num_features = so_training_h_sparse.shape[1]
num_features

### Computing the Bernoulli parameter

To compute the Bernoulli parameter we need a linear function, which needs weights and a bias. So here we obtain those.

Let's obtain $\boldsymbol\theta$ for binary classification using `num_features` input features.

In [ ]:
def init_params_linear1(num_features, key):
    """
    num_features: dimensionality of the feature space
    key: a JAX random generator

    Return w with shape [num_features] and bias with shape []
    """
    w = random.uniform(key, shape=(num_features,))
    b = random.uniform(key)
    return w, b

In [ ]:
key, param_key = random.split(key, 2) 
w, b = init_params_linear1(num_features, param_key)
assert w.shape == (num_features,)
assert b.shape == ()

**Exercise with solution** 

Implement a linear function of $\mathbf h(x)$ and $\boldsymbol\theta$ with a single real-valued output:

In [ ]:
def linear1(inputs, *, w, b):
    """
    inputs: a collection of inputs [batch_size, num_features]
      we normally program our models supporting the ability to perform the same operations
      for multiple documents at once
      we do so by "batching" documents together, so our first dimension is used 
      to iterate over different documents
    w: parameters (vector of size num_features)
      see that the parameters do not depend on batch size
      that's because parameters are a property of the model and batch size isn't
    b: parameters (single scalar)

    Return the following computation
        \sum_{d=1}^D x[d] * w[d] + b
    for each document x[n] amongst the inputs.
    """   
    raise NotImplementedError("Implement me!") 

As the dev set is not too large, we will convert it to a dense numpy array, so that we can use it within JAX.

In [ ]:
so_dev_h = so_dev_h_sparse.toarray()

# DO NOT use .toarray for the training data, you could run out of memory

Unlike sklearn, JAX does not understand string labels, so we must convert our labels to integers:

In [ ]:
so_training_y = label_as_int(so_training[:, 1])
so_dev_y = label_as_int(so_dev[:, 1])
so_test_y = label_as_int(so_test[:, 1])

Now we can test our `linear1` function:

In [ ]:
# We can give it a single document, then we get a single output
assert linear1(so_dev_h[0:1], w=w, b=b).shape == (1,)

In [ ]:
# or we can give it 10 documents, then we get 10 outputs
assert linear1(so_dev_h[0:10], w=w, b=b).shape == (10,)

In [ ]:
# or we can give it the entire dev set, and get one output per doc in the dev set
assert linear1(so_dev_h, w=w, b=b).shape == (so_dev_h.shape[0],)

<details>
    <summary><b>Solution</b></summary>

```python
def linear1(inputs, *, w, b):
    """
    inputs: a collection of inputs [batch_size, num_features]
      we normally program our models supporting the ability to perform the same operations
      for multiple documents at once
      we do so by "batching" documents together, so our first dimension is used 
      to iterate over different documents
    w: parameters (vector of size num_features)
      see that the parameters do not depend on batch size
      that's because parameters are a property of the model and batch size isn't
    b: parameters (single scalar)

    Return the following computation
        \sum_{d=1}^D x[d] * w[d] + b
    for each document x[n] amongst the inputs.
    """    
    # elementwise product
    # [batch_size, num_features]
    out = w * inputs  # by default this multiplication happens elementwise along the last dimension of the tensor
    # reduce the feature dimension via sum 
    # [batch_size]
    out = jnp.sum(out, axis=-1) 
    # add bias
    out = out + b
    # [batch_size]
    return out
```

---    
</details>

For this statistical model, a binary classifier, we need to map $x$ to a probability value (the Bernoulli parameter), but the linear function is not constrained accordingly:


In [ ]:
assert jnp.alltrue(linear1(so_dev_h, w=w, b=b) >= 0.) and jnp.alltrue(linear1(so_dev_h, w=w, b=b) <= 1.) == False

For that, jax offers the sigmoid function:

In [ ]:
def make_prob(linear_predictor):
    """
    Properly constrains the linear predictor in order to parameterise a Bernoulli distribution.

    linear_predictor: an array of linear predictors with shape [batch_size]
    """
    raise NotImplementedError("Implement me!")

which constrains everything as we wanted:

In [ ]:
assert jnp.alltrue(make_prob(linear1(so_dev_h, w=w, b=b)) >= 0.) and jnp.alltrue(make_prob(linear1(so_dev_h, w=w, b=b)) <= 1.) == True

<details>
    <summary><b>Solution</b></summary>

```python
def make_prob(linear_predictor):
    """Apply the sigmoid activation to the linear predictor"""
    return sigmoid(linear_predictor)  # this is a JAX function
```        
---    
</details>

**Exercise with solution** Assess the Bernoulli log pmf, make sure to use JAX differentiable code. Luckily for us JAX has a [`jax.scipy.sats`](https://jax.readthedocs.io/en/latest/jax.scipy.html#jax-scipy-stats) module, which contains many of our favourite distributions.

In [ ]:
def bernoulli_logpmf(y, p):
    """
    y: a batch of binary values with shape [batch_size]
    p: a batch of probability values with shape [batch_size]

    Return \log Bernoulli(y[n]|p[n]) for each element y[n] in the batch.
    """
    raise NotImplementedError("Implement me!")    

In [ ]:
assert jnp.allclose(jnp.exp(bernoulli_logpmf(np.array([1, 1, 1]), np.array([0.9, 0.5, 0.1]))), np.array([0.9, 0.5, 0.1]), 1e-6)
assert jnp.allclose(jnp.exp(bernoulli_logpmf(np.array([0, 0, 0]), np.array([0.9, 0.5, 0.1]))), np.array([0.1, 0.5, 0.9]), 1e-6)

<details>
    <summary><b>Solution</b></summary>

```python
def bernoulli_logpmf(y, p):
    """
    y: a batch of binary values with shape [batch_size]
    p: a batch of probability values with shape [batch_size]

    Return \log Bernoulli(y[n]|p[n]) for each element y[n] in the batch.
    """
    return jax.scipy.stats.bernoulli.logpmf(y, p=p)    
```
    
---    
</details>

<details>
    <summary><b>Check this, if you'd like to draw samples from Bernoulli in JAX</b></summary>

```python
y_sample = random.bernoulli(key, p=make_prob(linear1(bin_dev_h, w=w, b=b)))
```
---    
</details>


We can also check the performance of the most probable clas (for an untrained model):

In [ ]:
def bernoulli_mode(p):
    return jnp.where(p > 0.5, 1, 0)

In [ ]:
assert jnp.alltrue(bernoulli_mode(np.array([0.51, 0.1, 0.8])) == np.array([1, 0, 1]))

**Exercise with solution** Test the performance of the untrained Bernoulli GLM using the mode of the predicted Bernoulli distributions for the documents in the dev set.


<details>
    <summary><b>Solution</b></summary>


```python
# **SOLUTION**
y_best = bernoulli_mode(make_prob(linear1(so_dev_h, w=w, b=b)))
print(classification_report(so_dev[:,1], label_as_string(y_best), zero_division=0))
_ = ConfusionMatrixDisplay.from_predictions(so_dev[:,1], label_as_string(y_best))
```

---    
</details>

**Exercise with solution** Use this untrained model to plot the Bernoulli pmf for the 16 first documents in the dev set.

This model is not trained yet, so don't be surprised if the plots aren't interesting yet.

<details>
    <summary><b>Solution</b></summary>

```python
# **SOLUTION**

import textwrap 

fig, axs = plt.subplots(4, 4, sharex=True, figsize=(16, 16))

print("The plots show samples from the conditional model Y|X=x.\nThe red vertical line shows the observation Y=y for X=x.")

for i in range(16):
    x, y = so_dev[i, 0], so_dev[i, 1]
    h = so_ff.transform([x]).toarray()
    prob = make_prob(linear1(h, w=w, b=b)).item()

    _ = axs[i//4,i%4].set_title("\n".join(textwrap.wrap(x, 30)))
    _ = axs[i//4,i%4].plot(label_as_string([0, 1]), [1-prob, prob], 'x') 
    _ = axs[i//4,i%4].axvline(y, c='red')
fig.tight_layout(h_pad=1, w_pad=1)    
```

---    
</details>

### Parameter estimation

For a Bernoulli GLM, our likelihood function is:
\begin{equation}
\mathcal L_{\mathcal D}(\boldsymbol\theta) = \sum_{n=1}^N \log \mathrm{Bernoulli}(y^{(n)}|g(x^{(n)}; \boldsymbol \theta)) 
\end{equation}

And for stochastic gradient-based optimisation we will approximate it using mini batches.


In [ ]:
from tqdm.auto import tqdm
from collections import defaultdict

In [ ]:
def l2_regulariser(w, b):
    """Here we implement the L2 regulariser for you"""
    return jax.numpy.linalg.norm(w.flatten(), 2) + jax.numpy.linalg.norm(b.flatten(), 2)

In [ ]:
assert jnp.isclose(l2_regulariser(np.array([1., 2., 3.]), np.array(4.)), np.sqrt(1**2 + 2**2 + 3**2) + np.sqrt(4**2), 1e-3)

**Exercise with solution** Use [jax.scipy.stats](https://jax.readthedocs.io/en/latest/_autosummary/jax.scipy.stats.bernoulli.logpmf.html#jax.scipy.stats.bernoulli.logpmf) functions to define a *loss function* for the Bernoulli GLM model. Our loss function is the **negative** of the L2-regularised log-likelihood function.

In [ ]:
def bernoulli_loss(w, b, *, inputs, targets, l2weight=0.): 
    """
    w: weights of the linear model with shape [num_features]
    b: bias of the linear model with shape []
    inputs: h(x) with shape [batch_size, num_features]
    targets: y with shape [batch_size]
    l2weight: this is the lambda > 0 in the formula, the weight of the regulariser
    """
    raise NotImplementedError("Implement me!")

In [ ]:
test_case_inputs = np.array([[1., 0.], [0., 1.], [0., 0.]])
test_case_targets = np.array([1, 0, 1])
test_case_w = np.array([-1., 2.])
test_case_b = np.array(1.)
# the linear predictor is correct
assert jnp.allclose(linear1(test_case_inputs, w=test_case_w, b=test_case_b), np.array([0., 3., 1]), 1e-3), "Did you remember to compute dot-product with w and add the bias b?"
# the probability is correct
assert jnp.allclose(make_prob(np.array([0., 3., 1.])), np.array([0.5, 0.952574, 0.73105]), 1e-3), "Did you use the sigmoid function?"
# the regulariser is correct
assert jnp.isclose(l2_regulariser(test_case_w, test_case_b), np.array(3.23606), 1e-3), "Did you change the regulariser code?"
assert bernoulli_loss(test_case_w, test_case_b, inputs=test_case_inputs, targets=test_case_targets).shape == (), "Did you remember to take the mean?"
# the (unregulariser) loss function is correct
assert jnp.isclose(bernoulli_loss(test_case_w, test_case_b, inputs=test_case_inputs, targets=test_case_targets), - np.mean([np.log(0.5), np.log(1-0.952574), np.log(0.73105)]), 1e-3), "Did you remember the minus?"
# the regulariseed loss function is correct
assert jnp.isclose(bernoulli_loss(test_case_w, test_case_b, inputs=test_case_inputs, targets=test_case_targets, l2weight=1.0), - np.mean([np.log(0.5), np.log(1-0.952574), np.log(0.73105)]) + np.array(3.23606), 1e-3), "Did you remember the minus?"

<details>
    <summary><b>Solution</b></summary>

```python
def bernoulli_loss(w, b, *, inputs, targets, l2weight=0.): 
    """
    w: weights of the linear model with shape [num_features]
    b: bias of the linear model with shape []
    inputs: h(x) with shape [batch_size, num_features]
    targets: y with shape [batch_size]
    l2weight: this is the lambda > 0 in the formula, the weight of the regulariser
    """
    s = linear1(inputs, w=w, b=b)
    p = make_prob(s)
    l2 = l2_regulariser(w=w, b=b)
    return - jax.scipy.stats.bernoulli.logpmf(targets, p=p).mean(0) + l2weight * l2 
```

---    
</details>

Here we will pack the loss code for you in a way that can be used for automatic differentiation, and we will then compute the gradient with respect to $\mathbf w$ and $b$:

In [ ]:
from functools import partial  # partial allows us to fix some of the arguments of a function

loss_fn = partial(
    bernoulli_loss, # we want to fix some of the arguments of bernoulli_loss
    inputs=so_dev_h,   # namely, the part that concerns the observed data, both h(x)
    targets=so_dev_y,  # and y
    l2weight=0.     # we also want to fix the hyperparameters, 
) # this creates a loss function which is a function of w and b (exactly as we would like it to be)

# Now we tell JAX to evaluate the loss_fn for the parameter values (w, b) that we currently have
#  and compute partial derivatives for them
loss, (grad_w, grad_b) = value_and_grad(loss_fn, (0, 1))(w, b)
# the partial derivatives together form the gradient vector, which always has the same dimensionality as the parameter vector
assert grad_w.shape == (num_features,)
assert grad_b.shape == ()
# the loss, of course, is a scalar 
assert loss.shape == ()

print(loss.item())

This is it, you have all the ingredients needed to prescribe the model (its parameterisation in terms of $\mathbf h(x)$ and $\boldsymbol\theta$, to use it (i.e., compute log probability for a given $(x, y)$ pair), and to train it (i.e., estimate its paramters using regularised MLE).

We will wrap everything nicely for you into a single class, so that you can experiment with it.

# Python/JAX GLM Class

**Ungraded exercises** Study the class below, it implements a general purpose GLM for a conditional distribution with a 1-dimensional statistical parameter. This can be used, for example, for binary classification and for Poisson regression. This class lacks implementation for a couple of methods, but you should not attempt to implememtn it. Instead, those are implemented in classes that specialise this one. For example, study the BinaryClassifier class that we provide right after GLM1. That class, will complete the specfication of GLM1 by making it a Bernoulli conditional model.

In [ ]:
class GLM1:
    """
    This class contains all functionality that are shared across GLMs for single parameter distributions.
    Subclasses of this class must only implement the parts that are specific to their choice of distribution.
    For example, 
        GLM1 uses the mode for prediction, but how to compute the mode depends on which distribution we have.
        GLM1 knows that the activation function is needed in order to constrain the linear predictor correctly, 
         but the choice of activation function will depend on the distribution we have.        
    """

    def __init__(self, feature_function, seed=0):
        self.feature_function = feature_function        
        self.num_features = len(feature_function.get_feature_names_out())
        self.key = random.PRNGKey(seed)

    def sparse_h(self, x):
        """
        Map documents to scipy sparse features
        x: a list of documents, each a string

        Return scipy sparse matrix (from feature function).
        """
        return self.feature_function.transform(x)

    def feature_names(self):
        return self.feature_function.get_feature_names_out()
    
    def random_parameters(self):
        """
        Return randomly initialised weights and biases
        """
        w = random.uniform(self.key, shape=(self.num_features,))
        b = random.uniform(self.key) 
        return w, b

    def l2(self, w, b):
        """Compute the L2 regulariser"""
        return jax.numpy.linalg.norm(w.flatten(), 2) + jax.numpy.linalg.norm(b.flatten(), 2)

    def linear(self, inputs, *, w, b):
        """
        inputs: a collection of inputs [batch_size, num_features]
        we normally program our models supporting the ability to perform the same operations
        for multiple documents at once
        we do so by "batching" documents together, so our first dimension is used 
        to iterate over different documents
        w: parameters (vector of size num_features)
        see that the parameters do not depend on batch size
        that's because parameters are a property of the model and batch size isn't
        b: parameters (single scalar)

        Return x * w + b
        """    
        # elementwise product
        # [batch_size, num_features]
        out = w * inputs  # by default this multiplication happens elementwise along the last dimension of the tensor
        # reduce the feature dimension via sum 
        # [batch_size]
        out = jnp.sum(out, axis=-1) 
        # add bias
        out = out + b
        # [batch_size]
        return out

    def activation(self, linear_predictor):
        """
        Constrain the linear predictor

        linear_predictor: w*h(x)+b with shape [batch_size]
        """
        raise NotImplementedError("Implement me in a subclass")    

    def g(self, inputs, *, w, b):
        """
        Compute the statistical parameter of the conditional model.

        inputs: h(x) with shape [batch_size, num_features]
        w: weights with shape [num_features]
        b: bias with shape []

        Return g(x) with shape [batch_size]
        """
        linear_predictor = self.linear(inputs, w=w, b=b)
        parameter = self.activation(linear_predictor)
        return parameter

    def mode(self, g):
        """
        Return the mode of the distribution

        g: statistical parameter of the distribution with shape [batch_size]
        """
        raise NotImplementedError("Implement me in a subclass!")    

    def log_p(self, targets, g):
        """
        Return log probability mass (or density) of targets given the statistical parameter.

        targets: y with shape [batch_size]
        g: statistical parameter of the distribution with shape [batch_size]
        """
        raise NotImplementedError("Implement me in a subclass!")

    def loss(self, w, b, *, inputs, targets, l2weight=0.):
        """
        Compute the regularised negative log-likelihood of the model given observed (x, y) pairs.

        w: weights with shape [num_features]
        b: bias with shape []
        inputs: h(x) with shape [batch_size, num_features]
        targets: y with shape [batch_size]
        l2weight: contribution of the L2 regulariser

        Return - \sum_{n=1}^N P(Y=y[n]|X=x[n]) + L2(w,b)
        """
        return - self.log_p(targets, g=self.g(inputs, w=w, b=b)).mean(0) + l2weight * self.l2(w, b) 

    def predict(self, inputs, *, w, b):
        """
        Predict y for h using the mode of the conditional distribution.
        
        inputs: h(x) with shape [batch_size, num_features]
        w: weights with shape [num_features]
        b: bias with shape []

        Return \argmax_y P(Y=y|X=x)
        """
        return self.mode(self.g(inputs, w=w, b=b))

    def validate(self, inputs, targets, *, w, b):
        """
        Predict y for h and compare it to y_true using an appropriate metric.
        
        inputs: h(x) with shape [batch_size, num_features]
        targets: true targets with shape [batch_size]
        w: weights with shape [num_features]
        b: bias with shape []

        Return performance metric (e.g., f1 for classification, MAE for regression)
        """
        raise NotImplementedError("Implement me in a subclass!")

Here we specialise this class to the case of Binary classification with a Bernoulli conditional model.

In [ ]:
from sklearn.metrics import classification_report


class BinaryClassifier(GLM1):

    def __init__(self, feature_function, seed=0):
        super().__init__(feature_function, seed=seed)
        
    def activation(self, linear_predictor):
        """For Binary classification we need to parameterise a Bernoulli, thus we constrain the linear predictor using sigmoid"""
        return sigmoid(linear_predictor)

    def mode(self, g):
        """The mode of the Bernoulli is the outcome that receives more than 0.5 mass"""        
        return jnp.where(g > 0.5, 1, 0)

    def log_p(self, targets, g):
        """
        We use JAX scipy to return the log pmf of the Bernoulli
        """
        return jax.scipy.stats.bernoulli.logpmf(targets, p=g)

    def validate(self, inputs, targets, *, w, b):
        """For binary classification we report macro F1"""
        y_pred = self.predict(inputs, w=w, b=b)
        metrics = classification_report(targets, y_pred, output_dict=True, zero_division=0)
        return metrics['macro avg']['f1-score']

**Exercise with solution** Here we share some code useful for training a model. Study the steps of the training algorithm.

In [ ]:
from functools import partial


def get_batcher(data_size, batch_size, replace=False, rng=np.random.RandomState(1)):  
    """
    Return an iterable for indices of data points organised as batches of a given size (the last batch is potentially shorter).
    """  
    if rng is None:
        permutation = np.arange(data_size)
    else:
        permutation = rng.permutation(data_size)
    i = 0
    while i < data_size:
      yield permutation[i:i+batch_size]
      i += batch_size        


def train_model(
    model, training_h_sparse, training_y, dev_h_sparse, dev_y, 
    num_epochs=5, batch_size=500, validate_freq=10, 
    l2weight=1e-6, lr0=10., step_overwrite=0, log=None, 
    w=None, b=None, 
    rng=None):
    """
    model: an instance of GLM1 specialised to a type of distribution
    training_h_sparse: a scipy sparse matrix of features for the training data
    training_y: a numpy array of targets for the training data
    dev_h_sparse: a scipy sparse matrix of features for the validation data
    dev_y: a numpy array of targets for the validation data
    num_epochs: total number of passes over the entire training data
    batch_size: how many instances are used for a single gradient step        
    validate_freq: how often (in number of gradient steps) we compute performance of the validation set
    l2weight: the contribution of the L2 regulariser
    lr0: the initial learning rate    
    step_overwrite: when you continue training, you can overwrite the step number (which affects the schedule of the learning rate)
    log: when you continue training, you can reuse an existing log
    w: use this to continue training (instead of training from scratch)
    b: use this to continue training (instead of training from scratch)
    rng: numpy random number generator, use it for reproducibility
    """
    
    if w is None or b is None:  # we start with some random parameters
        w, b = model.random_parameters()
    
    dev_h = dev_h_sparse.toarray()
    data_size = training_h_sparse.shape[0]
    if log is None:
        log = defaultdict(list)
    
    # It's good to always run the model before training, 
    # just to catch any problems and to log its current performance
    log['val_metric'].append(model.validate(dev_h, dev_y, w=w, b=b))
    log['val_loss'].append(model.loss(w, b, inputs=dev_h, targets=dev_y, l2weight=l2weight))    
    
    # and we will train for a certain number of steps
    total_steps = num_epochs * len(list(get_batcher(data_size, batch_size)))
    step = step_overwrite  # and sometimes we have already trained for a bit  

    with tqdm(range(total_steps), desc='MLE') as bar:  # tqdm is a very cool progressbar :)
        # we pass over the entire data a number of times
        for epoch in range(num_epochs):            
            # but do so in random order, and process one mini batch at a time
            for ids in get_batcher(data_size, batch_size, rng=rng):

                # our learning rate decays with time, this is something needed for the theory of stochastic optimisation to check out
                lr = lr0/np.log10(10+step)  

                # we prepare our loss function
                # essentially, we fix the data it is based on (the inputs and targets in the mini batch)
                # and fix its hyperparameters
                # the only things that can vary are the parameters (w, b)
                loss_fn = partial(
                    model.loss, 
                    inputs=training_h_sparse[ids].toarray(),  # we need dense arrays but we better not store all dense arrays (else we run out of memory)
                    targets=training_y[ids], 
                    l2weight=l2weight
                )
                # here we tell JAX to evaluate the loss function at the point (w, b)
                # and also compute its partial derivatives at that point
                loss_value, (grad_w, grad_b) = value_and_grad(loss_fn, (0, 1))(w, b)
                
                # because we have a *loss* we will subtract the gradient (scaled by the learning rate)
                # from the current parameter values
                w -= lr * grad_w
                b -= lr * grad_b

                # Here we log some information for analysis later on
                log['loss'].append(loss_value.item())       
                if step % validate_freq == 0:  
                    log['val_metric'].append(model.validate(dev_h, dev_y, w=w, b=b))
                    log['val_loss'].append(model.loss(w, b, inputs=dev_h, targets=dev_y, l2weight=l2weight))
                bar.set_postfix({'epoch': epoch + 1, 'step': f"{step:4d}", 'lr': f"{lr:.4f}", 'loss': f"{loss_value:.4f}", 'val_loss': f"{log['val_loss'][-1]:.4f}", 'val_metric': log['val_metric'][-1]}) 
                bar.update()
                step += 1
    
    return (w, b), log

## Binary classifier experiment

From now on, we will use the following feature function. In NBC, we use counts, but counts are not a great idea for linear and generalised linear models. That's because raw counts vary with document length, which makes the magnitude of the dot product $\mathbf w^\top\mathbf h(x)$ also vary a lot with document length. 

It's better to take frequency information into account, we wil be using a tf-idf transformation of the raw counts, sklearn can do that for us.

In [ ]:
bin_ff = Pipeline(
    [
        ('vect', CountVectorizer(ngram_range=(1,1), min_df=5)), # we will be discarding tokens less frequent than 5
        ('tfidf', TfidfTransformer()),
    ]
)
bin_ff.fit(so_training[:, 0])

bin_training_h_sparse = bin_ff.transform(so_training[:, 0])
bin_training_y = label_as_int(so_training[:, 1])

bin_dev_h_sparse = bin_ff.transform(so_dev[:, 0])
bin_dev_y = label_as_int(so_dev[:, 1])

bin_test_h_sparse = bin_ff.transform(so_test[:, 0])
bin_test_y = label_as_int(so_test[:, 1])

bin_cls = BinaryClassifier(bin_ff)

In [ ]:
# This should take less than 10 minutes (the progress bar should give you a reliable estimate)
# for a faster run reduce num_epochs or increase batch_size 
# of course, do expect changes in performance
# you can use lr0 and l2weight to affect the optimiser and the objective function
(bin_w, bin_b), bin_log = train_model(
    bin_cls,
    bin_training_h_sparse,
    bin_training_y,
    bin_dev_h_sparse,
    bin_dev_y,
    lr0=5, # vary this to change the initial learning rate
    l2weight=1e-4, # vary this to control regularisation
    batch_size=100, # on GPU you can use larger batches
    num_epochs=10, # use more to train for longer
    rng=np.random.RandomState(42),
)

We should always visualise the training loss, the validation loss, and some performance metric. 

The validation loss is the log-likelihood of the model given the validation data only (not the training data). If that curve starts going up, while the training loss curve is going down, we detect overfitting (a situation where the model is memorising patterns that are specific to the training data and are of no help to classify heldout data). 

The metric for classification helps us see how the most probable class decision rule performs for heldout data.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 3))
_ = ax[0].plot(np.arange(len(bin_log['loss'])), bin_log['loss'], '.')
_ = ax[0].set_ylabel("Training loss")
_ = ax[0].set_xlabel("Steps")
_ = ax[1].plot(np.arange(len(bin_log['val_loss'])), bin_log['val_loss'], '.')
_ = ax[1].set_ylabel("Validation loss")
_ = ax[1].set_xlabel("Validation steps")
_ = ax[2].plot(np.arange(len(bin_log['val_metric'])), bin_log['val_metric'], '.')
_ = ax[2].set_ylabel("Validation macro F1")
_ = ax[2].set_xlabel("Validation steps")
fig.tight_layout(h_pad=1, w_pad=1)

We should always visualise the magnitude of the weights. If they get too large we typically have numerical instabilities and/or overfitting.

In [ ]:
_ = plt.hist(bin_w, bins='auto')

And we can have a look at the bias term:

In [ ]:
bin_b

We can also order the features by importance and inspect them

In [ ]:
print("20 features with large positive weight")
for f, w in sorted(zip(bin_cls.feature_names(), bin_w), key=lambda pair: pair[1], reverse=True)[:20]:
    print(f, w)

<a name="ex-bernoulli"> **Graded Exercise - Subjectivity classifier** 

Here you will experiment with various Bernoulli GLMs on the subjectivity dataset using the feature function we provided you with. Here we recommend 100 epochs, more can be better, less can be okay too. Without GPUs, more than 100 epochs may be too slow.

1. Train for 100 epochs with regularisation 1e-4. Plot the log information (training/validation loss and metric curves). Comment on when the classifier converged (when the curves got roughly flat), if at all. Make a good quality plot for full points.

2. Train for 100 epochs with and without regularisation (use l2weight of 0 and of 1e-4). Plot the log information (training/validation loss and metric curves). Also plot a histogram with the flattened vector of weights (no need for bias), compare the histograms for the variant with and without regularisation. Discuss whether you see any risk for overfitting with this model (it's not always the case that overfitting happens).

3. For your model with regularisation, assess the model on the test set (display a classification report and the confusion matrix. For that model, inspect some features that received large positive weights and large negative weights. Discuss whether the features you see there are plausibly related to subjectivity (large positive weight) and objectivity (large negative weight). 


Do not expect the performance to necessarily match the NBC. This dataset is simple enough that NBC is a *strong baseline*. Also, our feature function is not very complex, because we want to keep the notebook tutorial easy to use.


In [ ]:
# CONTRIBUTE YOUR OWN SOLUTION

**Optional extra** When you are done with the graded part of the assignment, try changing the feature function, see if you can affect the result in a meaningful way (esp, part 3 of the exercise).

## Poisson regressor experiment

Here you will develop a Poisson regressor. The Poisson regressor is useful when the target is a natural number (ordered and not easiliy bounded).

We created an artificial task for you (normally, we would prefer to give you a real task, for example, age detection using the [Blog authorship corpus](https://www.kaggle.com/rtatman/blog-authorship-corpus) by [Schler et al (2006)](http://www.cs.biu.ac.il/~schlerj/schler_springsymp06.pdf)), but we want to keep the tutorial lightweight so you can focus on learning). In this task we stripped the sentences in the subjectivity dataset of blank spaces, but recorded the number of space-separated tokens before we did so. The artificial task for you is: read a space-free document and predict a distribution over the number of tokens that it must have had originally.

For modelling device, use a GLM that parameterises a conditional Poisson model.


In [ ]:
num_tokens = json.load(gzip.open(urllib.request.urlopen("https://surfdrive.surf.nl/files/index.php/s/le13WJnWjDkY7MV/download")))
num_tokens.keys()

In [ ]:
def prepare_num_tokens_corpus(corpus):
    training_x, training_y = zip(*corpus['training'])
    training_y = np.array(training_y, dtype=int)
    dev_x, dev_y = zip(*corpus['dev'])
    dev_y = np.array(dev_y, dtype=int)
    test_x, test_y = zip(*corpus['test'])
    test_y = np.array(test_y, dtype=int)
    return training_x, training_y, dev_x, dev_y, test_x, test_y

In [ ]:
nt_training_x, nt_training_y, nt_dev_x, nt_dev_y, nt_test_x, nt_test_y = prepare_num_tokens_corpus(num_tokens)

<a name="ex-numtokens"> **Graded Exercise - Number of Tokens Data and Baseline**  As always when we encounter a new dataset, let's explore it.

1. For the training set, plot the length of $x$ in characters against its length in tokens $y$; you can use `plt.plot` or you can use something more interesting like [seaborn's jointplot](https://seaborn.pydata.org/generated/seaborn.jointplot.html). We assess the quality of your plots (eg, titled, labelled, with legends, visually clear, etc).

2. Inspect some examples, but don't invest too much time, this task should look trivial, albeit tedious, to a human.

3. Obtain a baseline based on linear regression from the feature $\mathrm{LengthInCharacters}(x)$ to the observed length in tokens $y$. For that use the training set. For linear regression you can use [scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html). This linear model will be our baseline. For a new input $x$, this linear model will predict a continuous number as number of tokens, to obtain an integer use *round*. Evaluate your baseline on the dev set using `mean_absolute_error` (MAE) from sklearn. Also plot true $y$ against the predicted value for the documents in the dev set.


The baseline MAE is between 2 and 3.

In [ ]:
# CONTRIBUTE YOUR OWN SOLUTION

For the Poisson regressor, we will be using the following feature function. It works at the character level and basically counts character ngrams from length 2 to 5, keeping only those reasonably frequent ones, and transforms counts using tf (but not idf).

In [ ]:
nt_ff = Pipeline(
    [
        ('vect', CountVectorizer(ngram_range=(2,5), min_df=10, analyzer='char')),
        ('tfidf', TfidfTransformer(use_idf=False)),
    ]
)

nt_ff.fit(nt_training_x)

In [ ]:
nt_training_h_sparse = nt_ff.transform(nt_training_x)
nt_training_h_sparse.shape

In [ ]:
nt_dev_h_sparse = nt_ff.transform(nt_dev_x)
nt_dev_h = nt_dev_h_sparse.toarray()
nt_dev_h_sparse.shape, nt_dev_h.shape

<a name="ex-poisson"> **Graded Exercise - Poisson regression**  You will now design a Poisson GLM. There are striking similarities with the Bernoulli GLM, and our class structure emphasises that, but be careful to implement a **Poisson** regressor here, and not just another Bernoulli model.

1. Complete the PoissonRegressor class, it only lacks implementation for two methods, namely, `activation` and `log_p`. To help you get this right, we have coded a few test cases that your implementation should pass.

2. Once you have it, train the model for 50 to 100 epochs (the more you can wait, the better) with regularisation 1e-4. Plot the information logged during training. 

3. For the regularised model, use the dev set to report the mean absolute error of your model against the baseline. Also plot the true length against the predicted length (as predicted by the baseline and as predicted by the Poisson model). There is a good chance that your baseline works better in terms of MAE, esp given our current feature function. Make some remarks about what you see (e.g., does your model work equally well on short and long documents?). 

4. Inspect the features that received higher positive weight and higher negative weight, see if you recognise sequences that look like word boundaries. Presumably, detecting word boundaries is useful for this task (the baseline cannot do that).

5. Finally, use your model to predict Poisson distributions for the 16 first documents in the dev set. For each Poisson, plot a histogram of samples from it, and using a vertical line also plot the observed value for $y$. Remember that once you've obtained the Poisson parameter from your GLM, you can use `np.random.poisson` to obtain samples, for example.



In [ ]:
from sklearn.metrics import mean_absolute_error


class PoissonRegressor(GLM1):

    def __init__(self, feature_function, seed=0):
        super().__init__(feature_function, seed=seed)
        
    def activation(self, linear_predictor):
        """In Poisson regression we constrain the linear predictor to being strictly positive, which can be done with exp or softplus, for example"""
        raise NotImplementedError("Implement me!")        

    def mode(self, g):        
        """The Poisson mode is obtained via floor"""
        return jnp.floor(g)

    def log_p(self, targets, g):
        """
        We use JAX scipy to return the log pmf of the Poisson
        """
        raise NotImplementedError("Implement me!")        

    def validate(self, inputs, targets, *, w, b):
        y_pred = self.predict(inputs, w=w, b=b)
        return mean_absolute_error(targets, y_pred) 

In [ ]:
poi_reg = PoissonRegressor(nt_ff)

In [ ]:
assert jnp.alltrue(poi_reg.activation(np.random.uniform(size=1000)) > 0.), "The Poisson parameter should be *strictly* positive (larger than 0)"

In [ ]:
test_case_targets = np.array([0, 1, 2, 3, 4])
test_case_rates = np.array([1., 1., 2., 2., 3.])
test_case_logpmf = np.array([-1.0000005, -1.       , -1.3068521, -1.7123183, -1.7836056])
assert jnp.allclose(poi_reg.log_p(test_case_targets, test_case_rates), test_case_logpmf, 1e-3), "Did you implement the correct expression for Poisson's log pmf? You can use JAX scipy code for that, rather than write it from scratch."

In [ ]:
# The progress bar gives you a good estimate, GPU makes it faster;
# for a faster run reduce num_epochs or increase batch_size 
# of course, do expect changes in performance
# you can use lr0 and l2weight to affect the optimiser and the objective function
(poi_w, poi_b), poi_log = train_model(
    poi_reg,
    nt_training_h_sparse,
    nt_training_y,
    nt_dev_h_sparse,
    nt_dev_y,
    lr0=5.,
    l2weight=1e-4,
    batch_size=100, # use more for faster training (if you are on GPU, this can be bigger)
    num_epochs=10, # use more to find better models
    rng=np.random.RandomState(42),
)

In [ ]:
# CONTRIBUTION YOUR OWN SOLUTION

**Optional extra** When you are done with the graded part of the assignment, try changing the feature function, see if you can affect the result in a meaningful way.